#Multimatch length > 2 su tutto

In [ ]:
import pandas as pd
import numpy as np
import multimatch_gaze as mg
import ast
from tqdm import tqdm
# Carica il file JSON nel DataFrame
df = pd.read_json('/content/gdrive/MyDrive/COCOSearch18-fixations-TP/coco_search18_fixations_TP_validation_split2.json')

In [ ]:
# Unisci i due dataframe lungo l'asse delle righe (asse 0)
#df= pd.concat([df_1, df_2], axis=0, ignore_index=True)

In [ ]:
# Calcola il numero di soggetti unici per ogni coppia di 'name' e 'task'
subjects_count = df.groupby(['name', 'task'])['subject'].nunique()

# Filtra le coppie che hanno esattamente 10 soggetti
couples_10_subjects = subjects_count[subjects_count == 10].index

# Filtra il dataframe originale per mantenere solo le coppie con 10 soggetti
df_filtered = df[df.set_index(['name', 'task']).index.isin(couples_10_subjects)]

# Filtra ulteriormente per mantenere solo le coppie in cui tutti i valori di 'length' > 2
df = df_filtered.groupby(['name', 'task']).filter(lambda x: (x['length'] > 2).all())

# Mostra il dataframe filtrato
df

,name,subject,task,condition,bbox,X,Y,T,length,correct,RT,split
0,000000211326.jpg,2,bottle,present,"[377, 0, 288, 213]","[848.3, 738.5, 1154.2, 532.0, 520.5]","[530.0, 360.3, 448.8, 90.6, 45.6]","[168, 157, 67, 208, 643]",5,1,920,valid
1,000000350270.jpg,2,bottle,present,"[315, 52, 150, 367]","[847.9, 434.3, 386.7, 374.7, 386.1]","[399.4, 235.5, 276.3, 280.2, 276.6]","[204, 240, 247, 211, 176]",5,1,1211,valid
3,000000328838.jpg,2,bottle,present,"[1292, 0, 239, 423]","[773.4, 763.8, 750.6, 1377.1]","[477.7, 223.1, 97.8, 184.7]","[170, 150, 135, 592]",4,1,1029,valid
5,000000026564.jpg,2,bottle,present,"[1414, 590, 134, 275]","[785.4, 661.6, 1051.7, 1459.6, 1516.0, 1452.1,...","[501.8, 626.0, 564.3, 709.7, 719.3, 669.7, 665.7]","[203, 134, 183, 334, 266, 119, 254]",7,1,1679,valid
6,000000418882.jpg,2,bottle,present,"[216, 242, 121, 364]","[805.0, 774.3, 1021.5, 303.2, 285.7]","[425.3, 587.4, 540.0, 480.5, 365.4]","[211, 200, 125, 166, 776]",5,1,1334,valid
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,000000086650.jpg,7,tv,present,"[1184, 659, 173, 173]","[726.2, 1001.6, 628.7, 1333.2, 1338.5]","[397.8, 439.7, 537.6, 696.1, 784.2]","[235, 337, 216, 217, 642]",5,1,1346,valid
3996,000000039963.jpg,7,tv,present,"[367, 413, 252, 193]","[818.5, 544.0, 497.5, 462.7, 439.0]","[429.4, 545.3, 626.1, 588.0, 626.3]","[239, 287, 145, 319, 326]",5,1,1342,valid
3999,000000309553.jpg,7,tv,present,"[144, 52, 623, 252]","[878.0, 977.6, 571.0, 485.3]","[437.5, 352.1, 234.7, 216.9]","[198, 55, 232, 581]",4,1,661,valid
4001,000000512240.jpg,7,tv,present,"[1243, 452, 341, 229]","[816.7, 1448.9, 1459.2]","[435.7, 617.2, 579.0]","[256, 173, 349]",3,1,632,valid


In [ ]:
check_subjects = df.groupby(['name', 'task'])['subject'].nunique()
print("Coppie (name, task) con numero di soggetti unici:")
print(check_subjects.value_counts())  # Dovrebbe contenere solo il valore 10


Coppie (name, task) con numero di soggetti unici:
subject
10    121
Name: count, dtype: int64


In [ ]:
check_length = (df['length'] > 2).all()
print(f"Tutti i valori di 'length' sono > 2? {check_length}")


Tutti i valori di 'length' sono > 2? True


In [ ]:
num_coppie_finale = df[['name', 'task']].drop_duplicates().shape[0]
print(f"Numero di coppie (name, task) nel dataset filtrato: {num_coppie_finale}")


Numero di coppie (name, task) nel dataset filtrato: 121


In [ ]:
print(df['length'].describe())

count    1210.000000
mean        4.747934
std         2.709863
min         3.000000
25%         3.000000
50%         4.000000
75%         5.000000
max        34.000000
Name: length, dtype: float64


In [ ]:
# Funzione per convertire stringhe di liste in array numpy
def convert_to_numpy(value):
    if isinstance(value, str):
        return np.array(ast.literal_eval(value))
    elif isinstance(value, list):
        return np.array(value)
    else:
        return np.array([])

# Creiamo le colonne "X", "Y", "T" convertite in numpy array
df['X'] = df['X'].apply(convert_to_numpy)
df['Y'] = df['Y'].apply(convert_to_numpy)
df['T'] = df['T'].apply(convert_to_numpy)


In [ ]:
df.shape

(1210, 12)

In [ ]:
# Funzione per calcolare la metrica MultiMatch
def multiMatch_metric(s1, s2, stimulus_width, stimulus_height):
    assert s1.shape[1] > 2 and s2.shape[1] > 2, "Scanpaths should be arrays of shape [nfix,4] to compute MultiMatch!"

    diff1 = (s1[:,3] - s1[:,2]) * 1000.
    ss1 = np.hstack([s1[:,0].reshape(-1,1), s1[:,1].reshape(-1,1), diff1.reshape(-1,1)])

    diff2 = (s2[:,3] - s2[:,2]) * 1000.
    ss2 = np.hstack([s2[:,0].reshape(-1,1), s2[:,1].reshape(-1,1), diff2.reshape(-1,1)])

    scan1_pd = pd.DataFrame({'start_x': ss1[:,0], 'start_y': ss1[:,1], 'duration': ss1[:,2]})
    scan2_pd = pd.DataFrame({'start_x': ss2[:,0], 'start_y': ss2[:,1], 'duration': ss2[:,2]})

    mm_scores = mg.docomparison(scan1_pd.to_records(), scan2_pd.to_records(), screensize=[stimulus_width, stimulus_height])

    return mm_scores

In [ ]:
from collections import defaultdict
import numpy as np
from tqdm import tqdm

gold_standard_dict = defaultdict(lambda: [np.nan] * 5)

# Iteriamo su ogni combinazione immagine-task
for (image, task), group in tqdm(df.groupby(['name', 'task']), desc="Elaborazione combinazioni immagine-task"):
    print(f"\nProcessing Image: {image}, Task: {task}")  # Debug: immagine e task corrente

    # Seleziona solo i soggetti da 1 a 10
    group_subjects = group[group['subject'].isin(range(1, 11))]
    scanpaths = []

    for subject_id in range(1, 11):
        subject_data = group_subjects[group_subjects['subject'] == subject_id]

        if subject_data.empty:
            print(f"  - Soggetto {subject_id} non ha dati validi.")
            continue

        print(f"  - Elaborazione soggetto {subject_id}, numero fissazioni: {len(subject_data)}")

        for idx, row in subject_data.iterrows():
            x = np.array(row['X'])
            y = np.array(row['Y'])
            durations = np.array(row['T'])

            # Controllo dimensioni
            if len(x) == len(y) == len(durations) and len(durations) > 0:
                start_times = np.cumsum(np.insert(durations, 0, 0))[:-1] / 1000.
                end_times = start_times + durations / 1000.

                if len(start_times) == len(end_times) == len(x):
                    scanpaths.append(np.column_stack([x, y, start_times, end_times]))
                    print(f"     Soggetto {subject_id}: {len(x)} fissazioni aggiunte.")
                else:
                    print(f"     Errore dimensioni per immagine {image}, task {task}, soggetto {subject_id}")
            else:
                print(f"     Dati inconsistenti per soggetto {subject_id} (X, Y, T non corrispondono).")

    # Se ci sono scanpaths validi, calcoliamo le similarità
    if scanpaths:
        similarity_scores = []
        print(f"  → Confronto tra {len(scanpaths)} scanpaths...")

        for i in range(len(scanpaths)):
            for j in range(i + 1, len(scanpaths)):
                score = multiMatch_metric(scanpaths[i], scanpaths[j], 1280, 720)
                similarity_scores.append(score)
                print(f"    ✓ Similarità tra soggetto {i+1} e {j+1}: {score}")

        # Se abbiamo calcolato delle similarità, prendiamo la mediana
        if similarity_scores:
            median_score = np.median(np.array(similarity_scores), axis=0)
            gold_standard_dict[(image, task)] = median_score
            print(f"   Similarità media salvata per {image} - {task}: {median_score}")
        else:
            print(f"   Nessuna similarità calcolata per {image} - {task}.")
    else:
        print(f"   Nessun scanpath valido per {image} - {task}.")


Elaborazione combinazioni immagine-task:   0%|          | 0/121 [00:00<?, ?it/s]


Processing Image: 000000006608.jpg, Task: keyboard
  - Elaborazione soggetto 1, numero fissazioni: 1
    → Soggetto 1: 5 fissazioni aggiunte.
  - Elaborazione soggetto 2, numero fissazioni: 1
    → Soggetto 2: 3 fissazioni aggiunte.
  - Elaborazione soggetto 3, numero fissazioni: 1
    → Soggetto 3: 6 fissazioni aggiunte.
  - Elaborazione soggetto 4, numero fissazioni: 1
    → Soggetto 4: 3 fissazioni aggiunte.
  - Elaborazione soggetto 5, numero fissazioni: 1
    → Soggetto 5: 5 fissazioni aggiunte.
  - Elaborazione soggetto 6, numero fissazioni: 1
    → Soggetto 6: 4 fissazioni aggiunte.
  - Elaborazione soggetto 7, numero fissazioni: 1
    → Soggetto 7: 4 fissazioni aggiunte.
  - Elaborazione soggetto 8, numero fissazioni: 1
    → Soggetto 8: 4 fissazioni aggiunte.
  - Elaborazione soggetto 9, numero fissazioni: 1
    → Soggetto 9: 3 fissazioni aggiunte.
  - Elaborazione soggetto 10, numero fissazioni: 1
    → Soggetto 10: 5 fissazioni aggiunte.
  → Confronto tra 10 scanpaths...
  

Elaborazione combinazioni immagine-task:   1%|          | 1/121 [00:00<00:21,  5.52it/s]

    ✓ Similarità tra soggetto 5 e 6: [0.8573143201562645, 0.7443450123227573, 0.8255671864298718, 0.7589322400244682, 0.9147465437788019]
    ✓ Similarità tra soggetto 5 e 7: [0.9017421179684728, 0.42969366948623444, 0.905942204341329, 0.8842621113873832, 0.47336834208552125]
    ✓ Similarità tra soggetto 5 e 8: [0.947507599928716, 0.9454842540526772, 0.9057721857583139, 0.88260633914451, 0.8989813242784381]
    ✓ Similarità tra soggetto 5 e 9: [0.8504190279009056, 0.2750598777350085, 0.9693958235110729, 0.7587160594710239, 0.6536144578313254]
    ✓ Similarità tra soggetto 5 e 10: [0.9469510974868512, 0.8136797543007442, 0.8993552667229279, 0.9191749233076476, 0.7434872566538135]
    ✓ Similarità tra soggetto 6 e 7: [0.8799939186711299, 0.722697952821491, 0.8408078346528971, 0.9170201047633664, 0.6594684385382059]
    ✓ Similarità tra soggetto 6 e 8: [0.8880860904467541, 0.8875214055019268, 0.8576043632236643, 0.8516531112452279, 0.9302011466329803]
    ✓ Similarità tra soggetto 6 e 9:

Elaborazione combinazioni immagine-task:   2%|▏         | 2/121 [00:00<00:22,  5.34it/s]

...

    ✓ Similarità tra soggetto 1 e 5: [0.9705601802671102, 0.9832214985922282, 0.9449984635463874, 0.8831575393604337, 0.430232558139535]
    ✓ Similarità tra soggetto 1 e 6: [0.957348126026086, 0.9477856021330079, 0.9194616165461887, 0.915044822212312, 0.5849056603773584]
    ✓ Similarità tra soggetto 1 e 7: [0.9142188416321234, 0.9583606667144142, 0.8310430008723863, 0.6944759254557625, 0.6150943396226415]
    ✓ Similarità tra soggetto 1 e 8: [0.9755053201312104, 0.8127101994137118, 0.9590113795921754, 0.9627091701532821, 0.5076964843219763]
    ✓ Similarità tra soggetto 1 e 9: [0.9682956252609133, 0.837515851727174, 0.9402585499428378, 0.935226012085207, 0.5151515151515151]
    ✓ Similarità tra soggetto 1 e 10: [0.9381595633439688, 0.5347405698131806, 0.9048756663319462, 0.7358692625596075, 0.6886792452830188]
    ✓ Similarità tra soggetto 2 e 3: [0.8669172866389531, 0.5433603370310189, 0.9190943576047557, 0.8918053635557566, 0.6242272771177434]
    ✓ Similarità tra soggetto 2 e 4: [

Elaborazione combinazioni immagine-task:  99%|█████████▉| 120/121 [00:21<00:00,  5.85it/s]

    ✓ Similarità tra soggetto 3 e 5: [0.9364891782065887, 0.6612181384408549, 0.9513924850533361, 0.8026313990261194, 0.7738574267262788]
    ✓ Similarità tra soggetto 3 e 6: [0.8848946535847794, 0.5024820638300209, 0.9044101124634527, 0.9163878797847808, 0.5991189427312772]
    ✓ Similarità tra soggetto 3 e 7: [0.967338746736192, 0.6844517244925035, 0.9583288101147173, 0.9462375368117195, 0.7679738562091503]
    ✓ Similarità tra soggetto 3 e 8: [0.9833140257729719, 0.9593989792564875, 0.9698081346517504, 0.9456369069791892, 0.6181818181818183]
    ✓ Similarità tra soggetto 3 e 9: [0.9761133479969061, 0.7340303528055601, 0.9722412373682725, 0.9755923286369841, 0.5363636363636364]
    ✓ Similarità tra soggetto 3 e 10: [0.9439536740470975, 0.7355480646661896, 0.8898280569029953, 0.8879988806223806, 0.9066666666666661]
    ✓ Similarità tra soggetto 4 e 5: [0.906221228538207, 0.579629926154551, 0.9416063689474965, 0.8661205085667062, 0.858596624793808]
    ✓ Similarità tra soggetto 4 e 6: 

Elaborazione combinazioni immagine-task: 100%|██████████| 121/121 [00:22<00:00,  5.49it/s]

    ✓ Similarità tra soggetto 5 e 8: [0.927044114687159, 0.6432404774369165, 0.8996010141257067, 0.8388534306829009, 0.6666666666666667]
    ✓ Similarità tra soggetto 5 e 9: [0.9360282373523365, 0.9672956263908296, 0.9231391708322765, 0.9064012141860383, 0.5336754170694828]
    ✓ Similarità tra soggetto 5 e 10: [0.9222818744072773, 0.45861978839655204, 0.9304077421840207, 0.9127027768231712, 0.7335746849021186]
    ✓ Similarità tra soggetto 6 e 7: [0.9264879139259303, 0.6837998079034288, 0.8890563501106608, 0.8569478459842496, 0.752808988764045]
    ✓ Similarità tra soggetto 6 e 8: [0.9533163897311919, 0.8025078589026988, 0.9381554565108742, 0.9357163272779548, 0.7976190476190474]
    ✓ Similarità tra soggetto 6 e 9: [0.9217343168574976, 0.7665558697146388, 0.9334877216946221, 0.8220299717864935, 0.6252403846153844]
    ✓ Similarità tra soggetto 6 e 10: [0.9082720220944668, 0.5255191810982611, 0.9684199924403575, 0.9050081508260022, 0.6090909090909091]
    ✓ Similarità tra soggetto 7 e

In [ ]:
#CALCOLO ORIGINALE CHE FUNZIONAVA, MA MENO EFFICIENTE A QUANTO PARE
'''
# Creare un dizionario per raccogliere i gold standard per ogni immagine e task
gold_standard_dict = {}

# Filtra per ogni combinazione di immagine e task
for (image, task), group in tqdm(df.groupby(['name', 'task']), desc="Elaborazione combinazioni immagine-task"):
    # Filtra solo i 10 soggetti
    group_subjects = group[group['subject'].isin(range(1, 11))]

    # Lista per gli scanpath di tutti i soggetti
    scanpaths = []

    # Crea gli scanpath per ogni soggetto
    for subject_id in range(1, 11):  # Itera sui 10 soggetti
        subject_data = group_subjects[group_subjects['subject'] == subject_id]

        for idx, row in subject_data.iterrows():
            x = np.array(row['X'])
            y = np.array(row['Y'])
            durations = np.array(row['T'])

            # Calcola il tempo di inizio e fine per ogni fissazione
            start_times = np.cumsum(np.insert(durations, 0, 0)) / 1000.  # Converte in secondi

            # Rimuovi lo 0 da start_times per avere la stessa lunghezza di durations
            start_times = np.delete(start_times, 0)  # Rimuovi il primo elemento

            # Verifica che le lunghezze siano uguali
            if len(x) == len(y) == len(durations):
                if len(start_times) == len(durations):
                    end_times = start_times + durations / 1000.  # La fine è il tempo di inizio + durata

                    # Crea gli scanpath con 4 colonne: x, y, start_time, end_time
                    scanpath = np.vstack([x, y, start_times, end_times]).T  # Matrice (n, 4)
                    scanpaths.append(scanpath)
                else:
                    print(f"Lunghezza di start_times e durations non corrispondente per il soggetto {row['subject']}")
            else:
                print(f"Errore nelle lunghezze delle fissazioni per il soggetto {row['subject']}")
                continue  # Ignora questa riga se le lunghezze non sono valide

    if len(scanpaths) > 0:
        similarity_scores = []
        for i in range(len(scanpaths)):
            for j in range(i + 1, len(scanpaths)):
                # Controlla se i due scanpath hanno la stessa lunghezza
                score = multiMatch_metric(scanpaths[i], scanpaths[j], 1280, 720)
                similarity_scores.append(score)

        similarity_scores = np.array(similarity_scores)
        if len(similarity_scores) > 0:
            gold_standard = np.median(similarity_scores, axis=0)
            gold_standard_dict[(image, task)] = gold_standard
        else:
            print(f"Nessuna similarità calcolata per {image} - {task}")
            gold_standard_dict[(image, task)] = [np.nan] * 5  # O un altro valore di default
    else:
        print(f"Nessun scanpath valido per {image} - {task}")
        gold_standard_dict[(image, task)] = [np.nan] * 5  # O un altro valore di default
'''

'\n# Creare un dizionario per raccogliere i gold standard per ogni immagine e task\ngold_standard_dict = {}\n\n# Filtra per ogni combinazione di immagine e task\nfor (image, task), group in tqdm(df.groupby([\'name\', \'task\']), desc="Elaborazione combinazioni immagine-task"):\n    # Filtra solo i 10 soggetti\n    group_subjects = group[group[\'subject\'].isin(range(1, 11))]\n\n    # Lista per gli scanpath di tutti i soggetti\n    scanpaths = []\n\n    # Crea gli scanpath per ogni soggetto\n    for subject_id in range(1, 11):  # Itera sui 10 soggetti\n        subject_data = group_subjects[group_subjects[\'subject\'] == subject_id]\n\n        for idx, row in subject_data.iterrows():\n            x = np.array(row[\'X\'])\n            y = np.array(row[\'Y\'])\n            durations = np.array(row[\'T\'])\n\n            # Calcola il tempo di inizio e fine per ogni fissazione\n            start_times = np.cumsum(np.insert(durations, 0, 0)) / 1000.  # Converte in secondi\n\n            # Rim

In [ ]:
# Creiamo il dataframe finale con i gold standard per ogni immagine e task
gold_standard_df = pd.DataFrame(
    [(image, task, *gold_standard_dict[(image, task)]) for (image, task) in gold_standard_dict],
    columns=['image', 'task', 'shape', 'direction', 'length', 'position', 'duration']
)

# Visualizza il dataframe con il gold standard
gold_standard_df

,image,task,shape,direction,length,position,duration
0,000000006608.jpg,keyboard,0.925012,0.744345,0.925071,0.900286,0.680851
1,000000009527.jpg,bottle,0.901444,0.623090,0.908698,0.835169,0.687576
2,000000024610.jpg,laptop,0.934607,0.713874,0.937902,0.905322,0.750706
3,000000026564.jpg,bottle,0.890363,0.866242,0.876146,0.874633,0.776596
4,000000034114.jpg,microwave,0.874777,0.771404,0.885033,0.866234,0.706977
...,...,...,...,...,...,...,...
116,000000560298.jpg,sink,0.948697,0.889917,0.927149,0.922023,0.703252
117,000000561783.jpg,oven,0.921682,0.577306,0.899983,0.875489,0.635762
118,000000564382.jpg,cup,0.939416,0.819592,0.924499,0.863121,0.704795
119,000000565034.jpg,car,0.918561,0.660840,0.931050,0.882813,0.620805


In [ ]:
nan_rows = gold_standard_df[gold_standard_df.isna().any(axis=1)]
nan_rows


,image,task,shape,length,direction,position,duration


In [ ]:
gold_standard_df_clean = gold_standard_df.dropna()
#gold_standard_df_clean = gold_standard_df.fillna(0)


In [ ]:
gold_standard_df_clean

,image,task,shape,length,direction,position,duration
0,000000006608.jpg,keyboard,0.925012,0.744345,0.925071,0.900286,0.680851
1,000000009527.jpg,bottle,0.901444,0.623090,0.908698,0.835169,0.687576
2,000000024610.jpg,laptop,0.934607,0.713874,0.937902,0.905322,0.750706
3,000000026564.jpg,bottle,0.890363,0.866242,0.876146,0.874633,0.776596
4,000000034114.jpg,microwave,0.874777,0.771404,0.885033,0.866234,0.706977
...,...,...,...,...,...,...,...
116,000000560298.jpg,sink,0.948697,0.889917,0.927149,0.922023,0.703252
117,000000561783.jpg,oven,0.921682,0.577306,0.899983,0.875489,0.635762
118,000000564382.jpg,cup,0.939416,0.819592,0.924499,0.863121,0.704795
119,000000565034.jpg,car,0.918561,0.660840,0.931050,0.882813,0.620805


In [ ]:
print("Originale:", gold_standard_df.shape)
print("Senza NaN:", gold_standard_df_clean.shape)
gold_standard_df.shape[0] - gold_standard_df_clean.shape[0]

Originale: (121, 7)
Senza NaN: (121, 7)


0

In [ ]:
stats_df = gold_standard_df.describe().loc[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]
stats_df

,shape,length,direction,position,duration
mean,0.926099,0.761559,0.915514,0.885470,0.698140
std,0.023624,0.087021,0.026911,0.042149,0.053363
min,0.856685,0.517706,0.822852,0.747275,0.518617
25%,0.911446,0.693804,0.900130,0.863121,0.666667
50%,0.924542,0.774314,0.919731,0.892029,0.699561
75%,0.942351,0.819592,0.933438,0.918565,0.733945
max,0.972769,0.923771,0.972321,0.948939,0.824612


In [ ]:
mean_values = stats_df.loc['mean']
mean_of_means = mean_values.mean()

# Mostra la media
print(mean_of_means)

0.8373564051574146


#Scan Match T length > 2 con T su tutti


In [ ]:
import pandas as pd
import numpy as np
import multimatch_gaze as mg
import ast
from tqdm import tqdm
# Carica il file JSON nel DataFrame
df = pd.read_json('/content/gdrive/MyDrive/COCOSearch18-fixations-TP/coco_search18_fixations_TP_train_split2.json')

In [ ]:
# Unisci i due dataframe lungo l'asse delle righe (asse 0)
#df= pd.concat([df_1, df_2], axis=0, ignore_index=True)

In [ ]:
# Calcola il numero di soggetti unici per ogni coppia di 'name' e 'task'
subjects_count = df.groupby(['name', 'task'])['subject'].nunique()

# Filtra le coppie che hanno esattamente 10 soggetti
couples_10_subjects = subjects_count[subjects_count == 10].index

# Filtra il dataframe originale per mantenere solo le coppie con 10 soggetti
df_filtered = df[df.set_index(['name', 'task']).index.isin(couples_10_subjects)]

# Filtra ulteriormente per mantenere solo le coppie in cui tutti i valori di 'length' > 2
df = df_filtered.groupby(['name', 'task']).filter(lambda x: (x['length'] > 2).all())

# Mostra il dataframe filtrato
df

,name,subject,task,condition,bbox,X,Y,T,length,correct,RT,split
0,000000211326.jpg,2,bottle,present,"[377, 0, 288, 213]","[848.3, 738.5, 1154.2, 532.0, 520.5]","[530.0, 360.3, 448.8, 90.6, 45.6]","[168, 157, 67, 208, 643]",5,1,920,valid
1,000000350270.jpg,2,bottle,present,"[315, 52, 150, 367]","[847.9, 434.3, 386.7, 374.7, 386.1]","[399.4, 235.5, 276.3, 280.2, 276.6]","[204, 240, 247, 211, 176]",5,1,1211,valid
3,000000328838.jpg,2,bottle,present,"[1292, 0, 239, 423]","[773.4, 763.8, 750.6, 1377.1]","[477.7, 223.1, 97.8, 184.7]","[170, 150, 135, 592]",4,1,1029,valid
5,000000026564.jpg,2,bottle,present,"[1414, 590, 134, 275]","[785.4, 661.6, 1051.7, 1459.6, 1516.0, 1452.1,...","[501.8, 626.0, 564.3, 709.7, 719.3, 669.7, 665.7]","[203, 134, 183, 334, 266, 119, 254]",7,1,1679,valid
6,000000418882.jpg,2,bottle,present,"[216, 242, 121, 364]","[805.0, 774.3, 1021.5, 303.2, 285.7]","[425.3, 587.4, 540.0, 480.5, 365.4]","[211, 200, 125, 166, 776]",5,1,1334,valid
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,000000086650.jpg,7,tv,present,"[1184, 659, 173, 173]","[726.2, 1001.6, 628.7, 1333.2, 1338.5]","[397.8, 439.7, 537.6, 696.1, 784.2]","[235, 337, 216, 217, 642]",5,1,1346,valid
3996,000000039963.jpg,7,tv,present,"[367, 413, 252, 193]","[818.5, 544.0, 497.5, 462.7, 439.0]","[429.4, 545.3, 626.1, 588.0, 626.3]","[239, 287, 145, 319, 326]",5,1,1342,valid
3999,000000309553.jpg,7,tv,present,"[144, 52, 623, 252]","[878.0, 977.6, 571.0, 485.3]","[437.5, 352.1, 234.7, 216.9]","[198, 55, 232, 581]",4,1,661,valid
4001,000000512240.jpg,7,tv,present,"[1243, 452, 341, 229]","[816.7, 1448.9, 1459.2]","[435.7, 617.2, 579.0]","[256, 173, 349]",3,1,632,valid


In [ ]:
# Funzione per convertire stringhe di liste in array numpy
def convert_to_numpy(value):
    if isinstance(value, str):
        return np.array(ast.literal_eval(value))
    elif isinstance(value, list):
        return np.array(value)
    else:
        return np.array([])

# Creiamo le colonne "X", "Y", "T" convertite in numpy array
df['X'] = df['X'].apply(convert_to_numpy)
df['Y'] = df['Y'].apply(convert_to_numpy)
df['T'] = df['T'].apply(convert_to_numpy)


In [ ]:
df.shape

(1210, 12)

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy


class ScanMatch(object):
    """
    ScanMatch Object.
    """
    def __init__(self, **kw):
        """
        :param int Xres:
        :param int Yres:
        :param int Xbin:
        :param int Ybin:
        :param float Threshold:
        :param float GapValue:
        :param float TempBin:
        :param (int, int) Offset:
        """
        self.Xres = 1024
        self.Yres = 768
        self.Xbin = 8
        self.Ybin = 6
        self.Threshold = 3.5
        self.GapValue = 0.0
        self.TempBin = 0.0
        self.Offset = (0, 0)

        for k in kw.keys():
            if k == 'Xres':
                self.Xres = kw[k]
            elif k == 'Yres':
                self.Yres = kw[k]
            elif k == 'Xbin':
                self.Xbin = kw[k]
            elif k == 'Ybin':
                self.Ybin = kw[k]
            elif k == 'Threshold':
                self.Threshold = kw[k]
            elif k == 'GapValue':
                self.GapValue = kw[k]
            elif k == 'TempBin':
                self.TempBin = kw[k]
            elif k == 'Offset':
                self.Offset = kw[k]
            else:
                raise ValueError('Unknown parameter: %s.' % k)

        self.intv = numpy.vectorize(int)

        self.createSubMatrix()
        self.gridMask()

    def createSubMatrix(self, Threshold=None):
        if Threshold is not None:
            self.Threshold = Threshold
        mat = numpy.zeros((self.Xbin*self.Ybin, self.Xbin*self.Ybin))
        indI = 0
        indJ = 0
        for i in range(self.Ybin):
            for j in range(self.Xbin):
                for ii in range(self.Ybin):
                    for jj in range(self.Xbin):
                        mat[indI, indJ] = numpy.sqrt((j-jj)**2 + (i-ii)**2)
                        indI += 1
                indI = 0
                indJ += 1
        max_sub = numpy.max(mat)
        self.SubMatrix = numpy.abs(mat-max_sub) - (max_sub - self.Threshold)

    def gridMask(self):
        a = numpy.reshape(numpy.arange(self.Xbin*self.Ybin), (self.Ybin, self.Xbin))
        m = float(self.Xbin) / self.Xres
        n = float(self.Ybin) / self.Yres
        xi = numpy.int32(numpy.arange(0, self.Xbin, m))
        yi = numpy.int32(numpy.arange(0, self.Ybin, n))

        self.mask = numpy.zeros((self.Yres, self.Xres))
        for y in range(self.Yres):
            self.mask[y, :] = a[yi[y], xi]

    def fixationToSequence(self, data):
        d = data.copy()
        d[:, :2] -= self.Offset
        d[d < 0] = 0
        d[d[:, 0] >= self.Xres, 0] = self.Xres-1
        d[d[:, 1] >= self.Yres, 1] = self.Yres-1
        d = self.intv(d)

        seq_num = self.mask[d[:, 1], d[:, 0]]

        if self.TempBin != 0:
            fix_time = numpy.round(d[:, 2] / float(self.TempBin))
            tmp = []
            for f in range(d.shape[0]):
                tmp.extend([seq_num[f] for x in range(int(fix_time[f]))])
            seq_num = numpy.array(tmp)

        return seq_num

    def match(self, A, B):
        n = len(A)
        m = len(B)

        F = numpy.zeros((n+1, m+1))
        for i in range(n+1):
            F[i, 0] = self.GapValue*(i+1)
        for j in range(m+1):
            F[0, j] = self.GapValue*(j+1)

        for i in range(1, n+1):
            for j in range(1, m+1):
                match = F[i-1, j-1] + self.SubMatrix[A[i-1], B[j-1]]
                delete = F[i-1, j] + self.GapValue
                insert = F[i, j-1] + self.GapValue
                F[i, j] = max([match, insert, delete])

        AlignmentA = numpy.zeros(n+m)-1
        AlignmentB = numpy.zeros(n+m)-1
        i = n
        j = m
        step = 0

        while(i > 0 and j > 0):
            score = F[i, j]
            scoreDiag = F[i-1, j-1]
            # scoreUp = F[i, j-1]
            scoreLeft = F[i-1, j]

            if score == scoreDiag + self.SubMatrix[A[i-1], B[j-1]]:
                AlignmentA[step] = A[i-1]
                AlignmentB[step] = B[j-1]
                i -= 1
                j -= 1
            elif score == scoreLeft + self.GapValue:
                AlignmentA[step] = A[i-1]
                i -= 1
            else:
                AlignmentB[step] = B[j-1]
                j -= 1

            step += 1

        while(i > 0):
            AlignmentA[step] = A[i-1]
            i -= 1
            step += 1

        while(j > 0):
            AlignmentB[step] = B[j-1]
            j -= 1
            step += 1

        F = F.transpose()

        maxF = numpy.max(F)
        maxSub = numpy.max(self.SubMatrix)
        scale = maxSub * max((m, n))
        matchScore = maxF / scale

        align = numpy.vstack([AlignmentA[step-1::-1], AlignmentB[step-1::-1]]).transpose()

        return matchScore, align, F

    def maskFromArray(self, array):
        self.mask = array

    def subMatrixFromArray(self, array):
        self.SubMarix = array


def generateMaskFromArray(data, threshold, margeColor):
    dataArray = data.copy()
    uniqueData = numpy.unique(dataArray)
    for i in range(len(uniqueData)):
        index = numpy.where(dataArray == uniqueData[i])
        if len(index[0]) <= threshold:
            dataArray[index] = margeColor

    uniqueData2 = numpy.unique(dataArray)
    for i in range(len(uniqueData2)):
        index = numpy.where(dataArray == uniqueData2[i])
        dataArray[index] = i

    return dataArray, uniqueData2

In [ ]:
def scanMatch_metric(s1, s2, stimulus_width, stimulus_height, Xbin=14, Ybin=8, TempBin=50):

    if s1.shape[1] > 2 and s2.shape[1] > 2:
        tb = TempBin
        diff1 = (s1[:,3] - s1[:,2]) * 1000.
        s1 = np.hstack([s1[:,0].reshape(-1,1), s1[:,1].reshape(-1,1), diff1.reshape(-1,1)])
        diff2 = (s2[:,3] - s2[:,2]) * 1000.
        s2 = np.hstack([s2[:,0].reshape(-1,1), s2[:,1].reshape(-1,1), diff2.reshape(-1,1)])

    elif s1.shape[1] == 2 and s2.shape[1] == 2:
        tb = 0.0
    else:
        raise ValueError("Scanpaths should be arrays of shape [nfix,2] or [nfix,4]! Columns are assumed to be x,y fixations coordinates and, eventually, their start and end times (seconds)")

    matchObject = ScanMatch(Xres=stimulus_width, Yres=stimulus_height, Xbin=Xbin, Ybin=Ybin, TempBin=TempBin)
    seq1 = matchObject.fixationToSequence(s1).astype(int)
    seq2 = matchObject.fixationToSequence(s2).astype(int)

    (score, _, _) = matchObject.match(seq1, seq2)

    return score

In [ ]:
from collections import defaultdict
import numpy as np
from tqdm import tqdm

gold_standard_dict = defaultdict(lambda: [np.nan] * 5)

for (image, task), group in tqdm(df.groupby(['name', 'task']), desc="Elaborazione combinazioni immagine-task"):
    group_subjects = group[group['subject'].isin(range(1, 11))]
    scanpaths = []

    for subject_id in range(1, 11):
        subject_data = group_subjects[group_subjects['subject'] == subject_id]

        for idx, row in subject_data.iterrows():
            x = np.array(row['X'])
            y = np.array(row['Y'])
            durations = np.array(row['T'])

            # Controlliamo che x, y e durations abbiano la stessa lunghezza
            if len(x) == len(y) == len(durations) and len(durations) > 0:
                start_times = np.cumsum(np.insert(durations, 0, 0))[:-1] / 1000.
                end_times = start_times + durations / 1000.

                # Controllo finale sulle dimensioni
                if len(start_times) == len(end_times) == len(x):
                    scanpaths.append(np.column_stack([x, y, start_times, end_times]))
                else:
                    print(f" Errore dimensioni per immagine {image}, task {task}, soggetto {subject_id}")

    if scanpaths:
        similarity_scores = []
        for i in range(len(scanpaths)):
            for j in range(i + 1, len(scanpaths)):

                score = scanMatch_metric(scanpaths[i], scanpaths[j], 1280, 720)
                similarity_scores.append(score)

        if similarity_scores:
            gold_standard_dict[(image, task)] = np.median(np.array(similarity_scores), axis=0)
        else:
            print(f" Nessuna similarità calcolata per {image} - {task}")
    else:
        print(f" Nessun scanpath valido per {image} - {task}")


Elaborazione combinazioni immagine-task: 100%|██████████| 121/121 [02:44<00:00,  1.36s/it]


In [ ]:
gold_standard_df = pd.DataFrame(
    [(image, task, gold_standard_dict[(image, task)]) for (image, task) in gold_standard_dict],
    columns=['image', 'task', 'ScanMatch']
)


In [ ]:
gold_standard_df

,image,task,ScanMatch
0,000000006608.jpg,keyboard,0.466667
1,000000009527.jpg,bottle,0.402042
2,000000024610.jpg,laptop,0.504202
3,000000026564.jpg,bottle,0.568540
4,000000034114.jpg,microwave,0.516220
...,...,...,...
116,000000560298.jpg,sink,0.465146
117,000000561783.jpg,oven,0.456775
118,000000564382.jpg,cup,0.384749
119,000000565034.jpg,car,0.476190


In [ ]:
nan_rows = gold_standard_df[gold_standard_df.isna().any(axis=1)]
nan_rows


,image,task,ScanMatch


In [ ]:
gold_standard_df_clean = gold_standard_df.dropna()
#gold_standard_df_clean = gold_standard_df.fillna(0)


In [ ]:
gold_standard_df_clean

,image,task,ScanMatch
0,000000006608.jpg,keyboard,0.466667
1,000000009527.jpg,bottle,0.402042
2,000000024610.jpg,laptop,0.504202
3,000000026564.jpg,bottle,0.568540
4,000000034114.jpg,microwave,0.516220
...,...,...,...
116,000000560298.jpg,sink,0.465146
117,000000561783.jpg,oven,0.456775
118,000000564382.jpg,cup,0.384749
119,000000565034.jpg,car,0.476190


In [ ]:
stats_df = gold_standard_df.describe()
stats_df

,ScanMatch
count,121.000000
mean,0.456856
std,0.099865
min,0.243182
25%,0.395179
50%,0.465146
75%,0.525171
max,0.702857
